<a href="https://colab.research.google.com/github/jintubhuyan-2000/ML-XAI_ForestFire/blob/main/Forest_RegionTransfer_California_FinalRevised2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gdown --quiet

# Use gdown to download the folder
!gdown "https://drive.google.com/drive/u/0/folders/17bKKw2k12s_ZxHLfAHPhyyxghZXI-mFZ" --folder

Retrieving folder contents
Processing file 1XxhtG6cFx9kNEaBkzDg9W6lyHw1nQnvE AccuracyStats_2025.csv
Processing file 1z6J3c-emQt0Skm9yu67-M16358O8XaKq ConfusionMatrix_2025.csv
Processing file 1DH1klTmk-Ev4ZMXDvUM8cu6mfGoe-62F FireProbStats_PerDistrict_forest.csv
Processing file 1eLfv4KjNlv8KV9ojecww39U8Y_umCfHE RF_RegionTransfer_Test_forest.csv
Processing file 1Zc4MgZfGD6hprmwRMcOWz0Y-rEs6ma9Y RF_SpatialCV_Test_forest.csv
Processing file 1KC-6mjAcWXP5ycuRb4Bpv6fqtELZEzML RF_TemporalSplit_Test2025.csv
Processing file 1bj7yXyCzvKMrlIwpyEl3ou01n-26BA1N TestPoints_Predictors_forest.csv
Processing file 1kf6Nome2XrQD3edOhNWuDkF4Vyy5nyGw TrainPoints_Predictors_forest.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1XxhtG6cFx9kNEaBkzDg9W6lyHw1nQnvE
To: /content/Validation Datasets/AccuracyStats_2025.csv
100% 126/126 [00:00<00:00, 494kB/s]
Downloading...
From: https://drive.google.com

In [6]:
#!/usr/bin/env python3
"""
Wildfire RF analysis with multiple runs & uncertainty.
Generates mean ± SD curves for ROC, PR, Calibration, Top-K.
Saves per-run metrics, Top-K captures, aggregated metrics by stratum, and summary.
"""

import os, glob, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, brier_score_loss,
                             roc_curve, precision_recall_curve, f1_score, precision_score, recall_score)
from sklearn.calibration import calibration_curve
import joblib

# Optional SHAP
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False
    warnings.warn('shap not available. Install shap to compute SHAP plots.')

# ----------------------
# Paths
# ----------------------
BASE_DIR = r"/content/Validation Datasets"
RESULTS_DIR = os.path.join('/content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5')
os.makedirs(RESULTS_DIR, exist_ok=True)

OUT_DIR = RESULTS_DIR
METRICS_PER_RUN_CSV = os.path.join(OUT_DIR, "metrics_per_run.csv")
METRICS_AGG_CSV = os.path.join(OUT_DIR, "metrics_aggregated_by_stratum.csv")
TOPK_PER_RUN_CSV = os.path.join(OUT_DIR, "topk_per_run.csv")
TOPK_AGG_CSV = os.path.join(OUT_DIR, "topk_aggregated.csv")
SUMMARY_TXT = os.path.join(OUT_DIR, "summary_report_multi_run.txt")

# ----------------------
# CSV detection
# ----------------------
csv_files = glob.glob(os.path.join(BASE_DIR, '**', '*.csv'), recursive=True)
def find_candidate(files, keywords):
    keywords = [k.lower() for k in keywords]
    for f in files:
        name = os.path.basename(f).lower()
        if any(k in name for k in keywords):
            return f
    return None

train_csv = find_candidate(csv_files, ['train'])
test_csv = find_candidate(csv_files, ['test'])
if not train_csv or not test_csv:
    csv_sizes = sorted(csv_files, key=os.path.getsize, reverse=True)
    if len(csv_sizes) >= 2:
        train_csv = csv_sizes[0] if not train_csv else train_csv
        test_csv = csv_sizes[1] if not test_csv else test_csv

print('Train CSV:', train_csv)
print('Test CSV:', test_csv)

train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

# Clean column names
train.columns = [c.strip() for c in train.columns]
test.columns = [c.strip() for c in test.columns]

# Target column
possible_targets = ['class', 'y', 'label', 'fire', 'is_fire']
train_cols_low = [c.lower() for c in train.columns]
target_col = next((train.columns[i] for i,t in enumerate(train_cols_low) if t in possible_targets), None)
if target_col is None:
    raise ValueError("Target column not found")

train[target_col] = train[target_col].astype(int)
if target_col in test.columns:
    test[target_col] = test[target_col].astype(int)

y_train = train[target_col].values
y_true = test[target_col].values if target_col in test.columns else None

# Predictor columns
predictors = [c for c in train.select_dtypes(include=[np.number]).columns
              if c != target_col and 'id' not in c.lower()]
X_train = train[predictors].fillna(-999)
X_test = test[predictors].fillna(-999)

# ----------------------
# Multiple runs
# ----------------------
NUM_RUNS = 10
SEEDS = list(range(42, 42+NUM_RUNS))

# Storage
roc_list, pr_list, cal_list, topk_list = [], [], [], []
metrics_all = []
y_prob_runs = []

for seed in SEEDS:
    rf = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1)
    rf.fit(X_train, y_train)
    joblib.dump(rf, os.path.join(RESULTS_DIR, f'rf_model_{seed}.joblib'))

    y_prob = rf.predict_proba(X_test)[:,1]
    y_prob_runs.append(y_prob)
    y_pred = (y_prob >= 0.5).astype(int)

    # Metrics
    metrics = {
        'seed': seed,
        'roc_auc': roc_auc_score(y_true, y_prob),
        'pr_auc': average_precision_score(y_true, y_prob),
        'brier': brier_score_loss(y_true, y_prob),
        'f1': f1_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred)
    }
    metrics_all.append(metrics)

    # ROC
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_list.append(np.interp(np.linspace(0,1,100), fpr, tpr))

    # PR
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    pr_list.append(np.interp(np.linspace(0,1,100), recall[::-1], precision[::-1]))

    # Calibration
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)
    cal_list.append(np.interp(np.linspace(0,1,10), prob_pred, prob_true))

    # Top-K capture
    df_test = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob}).sort_values('y_prob', ascending=False)
    total_pos = df_test['y_true'].sum()
    topk_capture = [df_test.iloc[:int(np.ceil(len(df_test)*f))]['y_true'].sum()/total_pos
                    for f in np.linspace(0.01,1,100)]
    topk_list.append(topk_capture)

# Convert to arrays
roc_arr = np.array(roc_list)
pr_arr = np.array(pr_list)
cal_arr = np.array(cal_list)
topk_arr = np.array(topk_list)
metrics_df = pd.DataFrame(metrics_all)

# ----------------- Plotting with ribbons -----------------
ROC_FPR_GRID = np.linspace(0,1,100)
PR_RECALL_GRID = np.linspace(0,1,100)
TOPK_PERCENTS = np.linspace(0.01,1,100)
CAL_PROB_BINS = np.linspace(0,1,11)  # 10 bins

# ---------- ROC Curve ----------
mean_tpr = np.nanmean(roc_arr, axis=0)
sd_tpr = np.nanstd(roc_arr, axis=0, ddof=1)
plt.figure(figsize=(6,5))
plt.plot(ROC_FPR_GRID, mean_tpr, label=f"Mean ROC (n={NUM_RUNS})")
plt.fill_between(ROC_FPR_GRID, mean_tpr - sd_tpr, mean_tpr + sd_tpr, alpha=0.2)
plt.plot([0,1],[0,1], linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (mean ± SD)')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'roc_curve_mean_sd.png'), dpi=200)
plt.close()

# ---------- PR Curve ----------
mean_prec = np.nanmean(pr_arr, axis=0)
sd_prec = np.nanstd(pr_arr, axis=0, ddof=1)
plt.figure(figsize=(6,5))
plt.plot(PR_RECALL_GRID, mean_prec, label=f"Mean PR (n={NUM_RUNS})")
plt.fill_between(PR_RECALL_GRID, mean_prec - sd_prec, mean_prec + sd_prec, alpha=0.2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (mean ± SD)')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'pr_curve_mean_sd.png'), dpi=200)
plt.close()

# ---------- Top-K Capture ----------
mean_topk = np.nanmean(topk_arr, axis=0)
sd_topk = np.nanstd(topk_arr, axis=0, ddof=1)
plt.figure(figsize=(6,4))
plt.plot(TOPK_PERCENTS, mean_topk, marker='o', label='Mean Top-K capture')
plt.fill_between(TOPK_PERCENTS, mean_topk - sd_topk, mean_topk + sd_topk, alpha=0.2)
plt.xlabel('Top-k percent of highest risk area')
plt.ylabel('Fraction of fires captured')
plt.title('Top-K Capture (mean ± SD)')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'topk_curve_mean_sd.png'), dpi=200)
plt.close()

# ---------- Reliability / Calibration ----------
cal_bin_centers = (CAL_PROB_BINS[:-1] + CAL_PROB_BINS[1:]) / 2.0
mean_bin_obs = np.nanmean(cal_arr, axis=0)
sd_bin_obs = np.nanstd(cal_arr, axis=0, ddof=1)
plt.figure(figsize=(6,5))
plt.plot(cal_bin_centers, mean_bin_obs, marker='o', label='Mean calibration')
plt.fill_between(cal_bin_centers, mean_bin_obs - sd_bin_obs, mean_bin_obs + sd_bin_obs, alpha=0.2)
plt.plot([0,1],[0,1], linestyle='--', label='Perfect')
plt.xlabel('Predicted probability (bin center)')
plt.ylabel('Observed frequency')
plt.title('Reliability Diagram (mean ± SD)')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'reliability_mean_sd.png'), dpi=200)
plt.close()

# ----------------- Save per-run metrics -----------------
metrics_df.to_csv(METRICS_PER_RUN_CSV, index=False)
topk_df = pd.DataFrame(topk_arr, columns=TOPK_PERCENTS)
topk_df['seed'] = SEEDS
topk_df.to_csv(TOPK_PER_RUN_CSV, index=False)
print(f"Saved per-run metrics: {METRICS_PER_RUN_CSV}")
print(f"Saved per-run Top-K captures: {TOPK_PER_RUN_CSV}")

# ----------------- Aggregate metrics by stratum -----------------
if 'stratum' in test.columns:
    stratum_vals = test['stratum'].unique()
    agg_metrics_list = []
    agg_topk_list = []

    for s in stratum_vals:
        idx = test[test['stratum']==s].index
        metrics_sub, topk_sub = [], []

        for seed_i, seed in enumerate(SEEDS):
            y_prob_sub = y_prob_runs[seed_i][idx]
            y_true_sub = y_true[idx]
            y_pred_sub = (y_prob_sub >= 0.5).astype(int)

            metrics_sub.append({
                'seed': seed,
                'stratum': s,
                'roc_auc': roc_auc_score(y_true_sub, y_prob_sub),
                'pr_auc': average_precision_score(y_true_sub, y_prob_sub),
                'brier': brier_score_loss(y_true_sub, y_prob_sub),
                'f1': f1_score(y_true_sub, y_pred_sub),
                'precision': precision_score(y_true_sub, y_pred_sub),
                'recall': recall_score(y_true_sub, y_pred_sub)
            })

            df_sub = pd.DataFrame({'y_true': y_true_sub, 'y_prob': y_prob_sub}).sort_values('y_prob', ascending=False)
            total_pos_sub = df_sub['y_true'].sum()
            topk_capture_sub = [df_sub.iloc[:int(np.ceil(len(df_sub)*f))]['y_true'].sum()/total_pos_sub
                                for f in TOPK_PERCENTS]
            topk_sub.append(topk_capture_sub)

        metrics_sub_df = pd.DataFrame(metrics_sub)
        agg_metrics_list.append(metrics_sub_df.groupby('stratum').agg(['mean','std']))

        topk_sub_arr = np.array(topk_sub)
        topk_mean = np.nanmean(topk_sub_arr, axis=0)
        topk_sd = np.nanstd(topk_sub_arr, axis=0, ddof=1)
        agg_topk_list.append(pd.DataFrame({'stratum': s, 'topk_percent': TOPK_PERCENTS,
                                          'mean_capture': topk_mean, 'sd_capture': topk_sd}))

    pd.concat(agg_metrics_list).to_csv(METRICS_AGG_CSV)
    pd.concat(agg_topk_list).to_csv(TOPK_AGG_CSV, index=False)
    print(f"Saved aggregated metrics by stratum: {METRICS_AGG_CSV}")
    print(f"Saved aggregated Top-K by stratum: {TOPK_AGG_CSV}")
else:
    print("No 'stratum' column found in test CSV; skipping aggregation by stratum.")

# ----------------- Summary -----------------
with open(SUMMARY_TXT,'w') as fh:
    fh.write(f'Random Forest multiple runs evaluation ({NUM_RUNS} runs)\n')
    fh.write('===============================\n')
    fh.write(f'Train CSV: {train_csv}\n')
    fh.write(f'Test CSV: {test_csv}\n')
    fh.write(f'Seed list: {SEEDS}\n\n')
    fh.write('Metrics (mean ± SD):\n')
    mean_metrics = metrics_df.mean()
    sd_metrics = metrics_df.std()
    for col in metrics_df.columns:
        if col != 'seed':
            fh.write(f' - {col}: {mean_metrics[col]:.4f} ± {sd_metrics[col]:.4f}\n')
    fh.write('\nAll plots saved in the results directory.\n')
print(f"Summary report written: {SUMMARY_TXT}")

print('\nAll done. Check results in:', RESULTS_DIR)


Train CSV: /content/Validation Datasets/TrainPoints_Predictors_forest.csv
Test CSV: /content/Validation Datasets/RF_TemporalSplit_Test2025.csv
Saved per-run metrics: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5/metrics_per_run.csv
Saved per-run Top-K captures: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5/topk_per_run.csv
No 'stratum' column found in test CSV; skipping aggregation by stratum.
Summary report written: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5/summary_report_multi_run.txt

All done. Check results in: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5


In [7]:
# Wildfire RF Evaluation & SHAP Analysis (multi-run + uncertainty)
# Paste into a Jupyter cell. Requires: pandas, numpy, matplotlib, scikit-learn
# Optional: shap (pip install shap) for SHAP outputs.
# Outputs: metrics CSV (per-run + aggregated), topk CSV, curves, summary text.
# Author: adapted for multi-run uncertainty & reproducibility

import os
from pathlib import Path
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_auc_score, roc_curve,
    average_precision_score, precision_recall_curve,
    brier_score_loss, f1_score, precision_score, recall_score
)
from sklearn.calibration import calibration_curve

# Optional SHAP
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

# ----------------- CONFIG -----------------
search_dirs = [
    r"/content/Validation Datasets",
]
OUT_DIR = Path(r"//content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# resampling / reproducibility settings
K_RUNS = 10
BASE_SEED = 20250908
SAMPLING_METHOD = "stratified_bootstrap"
TEST_SAMPLE_FRACTION = 1.0

# Settings for curves / interpolation
ROC_FPR_GRID = np.linspace(0,1,200)
PR_RECALL_GRID = np.linspace(0,1,200)
CAL_PROB_BINS = np.linspace(0.0, 1.0, 11)
TOPK_PERCENTS = [1,5,10,20,30,40,50]

# Filenames
METRICS_PER_RUN_CSV = OUT_DIR / "metrics_per_run.csv"
METRICS_AGG_CSV = OUT_DIR / "metrics_aggregated_by_stratum.csv"
TOPK_PER_RUN_CSV = OUT_DIR / "topk_per_run.csv"
TOPK_AGG_CSV = OUT_DIR / "topk_aggregated.csv"
SUMMARY_TXT = OUT_DIR / "summary_report_multi_run.txt"

# ----------------- Helpers -----------------
def find_candidate_csv(dirs):
    candidates = []
    for d in dirs:
        for p in Path(d).rglob("*.csv"):
            candidates.append(p)
    if not candidates:
        return None
    prioritized = [p for p in candidates if any(k in p.name.lower() for k in ("test2025","test_2025","rf_regiontransfer","rf_spatialcv_test","rf_region_transfer_test","rf_spatialcv_test","rf_"))]
    if prioritized:
        return prioritized[0]
    prioritized = [p for p in candidates if any(k in p.name.lower() for k in ("test","2025","regiontransfer","region_transfer"))]
    if prioritized:
        return prioritized[0]
    return candidates[0]

def topk_capture(y_true, y_prob, ks=TOPK_PERCENTS):
    out = []
    order = np.argsort(-y_prob)
    y_true_sorted = np.array(y_true)[order]
    total_pos = float(y_true_sorted.sum())
    n = len(y_true_sorted)
    for k in ks:
        frac = k / 100.0
        top_n = max(1, int(math.ceil(n * frac)))
        captured = int(y_true_sorted[:top_n].sum())
        capture_rate = (captured / total_pos) if total_pos > 0 else np.nan
        out.append({
            "top_%": k,
            "top_n": top_n,
            "pos_captured_count": captured,
            "pos_captured_frac": capture_rate
        })
    return pd.DataFrame(out)

def interpolate_curve(x, y, x_grid):
    xp = np.clip(x, 0.0, 1.0)
    yp = np.clip(y, 0.0, 1.0)
    xp_unique, idx = np.unique(xp, return_index=True)
    yp_unique = yp[idx]
    if len(xp_unique) < 2:
        return np.full_like(x_grid, yp_unique[0] if len(yp_unique)>0 else np.nan)
    return np.interp(x_grid, xp_unique, yp_unique)

# ----------------- Locate CSV -----------------
csv_path = find_candidate_csv(search_dirs)
if csv_path is None:
    raise FileNotFoundError(f"No CSV found in {search_dirs}. Place your exported CSV(s) in one of those folders.")
print("Using CSV:", csv_path)

# ----------------- Load & detect columns -----------------
df_orig = pd.read_csv(csv_path)
cols = list(df_orig.columns)
y_true_col = None
y_prob_col = None

for c in cols:
    lc = c.lower().strip()
    if lc in ('class','label','y','ground_truth','is_fire','fire') and y_true_col is None:
        y_true_col = c
    if lc in ('classification','probability','prob','pred','pred_prob','probability_1') and y_prob_col is None:
        y_prob_col = c
# fallback heuristics
if y_true_col is None:
    for c in cols:
        if 'class' in c.lower() or c.lower().startswith('label'):
            y_true_col = c
            break
if y_prob_col is None:
    for c in cols:
        if any(k in c.lower() for k in ('prob','classification','pred')):
            y_prob_col = c
            break
if y_true_col is None or y_prob_col is None:
    raise ValueError(f"Could not auto-detect required columns. Found: {cols}")
print("Detected columns -> label:", y_true_col, ", prob:", y_prob_col)

# optional stratum column detection
stratum_col = None
for candidate in ['landcover','land_cover','lc','class_name','stratum','habitat','lcc']:
    if candidate in [c.lower() for c in cols]:
        for c in cols:
            if c.lower()==candidate:
                stratum_col = c
                break
        break
if stratum_col:
    print("Detected stratum column:", stratum_col)
else:
    print("No stratum column detected - will evaluate overall only.")

# ----------------- Data cleaning -----------------
df_orig = df_orig.dropna(subset=[y_true_col, y_prob_col]).copy()
df_orig['y_true'] = df_orig[y_true_col].astype(int)
df_orig['y_prob'] = pd.to_numeric(df_orig[y_prob_col], errors='coerce').clip(0,1)
df_orig = df_orig.dropna(subset=['y_prob']).reset_index(drop=True)
n_total = len(df_orig)
pos_rate_total = df_orig['y_true'].mean()

# ----------------- Multi-run evaluation -----------------
rng = np.random.default_rng(BASE_SEED)
seeds = [int(r) for r in rng.integers(0, 2**31-1, size=K_RUNS)]
print(f"Running {K_RUNS} runs with seeds: {seeds}")

metrics_rows = []
topk_rows = []

roc_tpr_matrix = np.zeros((K_RUNS, len(ROC_FPR_GRID)))
pr_prec_matrix = np.zeros((K_RUNS, len(PR_RECALL_GRID)))
cal_bin_obs_matrix = np.zeros((K_RUNS, len(CAL_PROB_BINS)-1))
topk_matrix = np.zeros((K_RUNS, len(TOPK_PERCENTS)))

for run_idx, seed in enumerate(seeds):
    np.random.seed(seed)
    # ----------------- Sampling -----------------
    if SAMPLING_METHOD == 'stratified_bootstrap':
        idx_list = []
        for cls in df_orig['y_true'].unique():
            cls_idx = df_orig.index[df_orig['y_true']==cls].tolist()
            if len(cls_idx)==0:
                continue
            s = np.random.choice(cls_idx, size=len(cls_idx), replace=True)
            idx_list.extend(s.tolist())
        sampled_idx = np.array(idx_list)
    elif SAMPLING_METHOD == 'subsample_no_replacement':
        n_take = int(math.ceil(n_total * TEST_SAMPLE_FRACTION))
        sampled_idx = np.random.choice(df_orig.index, size=n_take, replace=False)
    df = df_orig.loc[sampled_idx].reset_index(drop=True)

    # ----------------- Metrics -----------------
    try:
        roc_auc = roc_auc_score(df['y_true'], df['y_prob'])
    except Exception: roc_auc = np.nan
    try:
        pr_auc = average_precision_score(df['y_true'], df['y_prob'])
    except Exception: pr_auc = np.nan
    brier = brier_score_loss(df['y_true'], df['y_prob'])
    thresh = 0.5
    y_pred = (df['y_prob'] >= thresh).astype(int)
    f1 = f1_score(df['y_true'], y_pred, zero_division=0)
    prec = precision_score(df['y_true'], y_pred, zero_division=0)
    rec = recall_score(df['y_true'], y_pred, zero_division=0)
    metrics_rows.append({
        "run": run_idx, "seed": seed, "n_samples": len(df),
        "positive_rate": float(df['y_true'].mean()),
        "roc_auc": float(roc_auc), "pr_auc": float(pr_auc),
        "brier": float(brier), "threshold": thresh,
        "f1_at_0.5": float(f1),
        "precision_at_0.5": float(prec),
        "recall_at_0.5": float(rec)
    })

    # ----------------- Curves -----------------
    try: fpr, tpr, _ = roc_curve(df['y_true'], df['y_prob']); tpr_interp = interpolate_curve(fpr, tpr, ROC_FPR_GRID)
    except Exception: tpr_interp = np.full_like(ROC_FPR_GRID, np.nan)
    roc_tpr_matrix[run_idx,:] = tpr_interp

    try: precision, recall, _ = precision_recall_curve(df['y_true'], df['y_prob'])
    except Exception: precision, recall = np.array([]), np.array([])
    if len(recall)>1:
        recall_sort_idx = np.argsort(recall)
        prec_interp = interpolate_curve(recall[recall_sort_idx], precision[recall_sort_idx], PR_RECALL_GRID)
    else:
        prec_interp = np.full_like(PR_RECALL_GRID, np.nan)
    pr_prec_matrix[run_idx,:] = prec_interp

    try:
        df['prob_bin'] = pd.cut(df['y_prob'], bins=CAL_PROB_BINS, include_lowest=True, labels=False)
        obs_per_bin = [df.loc[df['prob_bin']==b, 'y_true'].mean() if (df['prob_bin']==b).sum()>0 else np.nan
                       for b in range(len(CAL_PROB_BINS)-1)]
        cal_bin_obs_matrix[run_idx,:] = np.array(obs_per_bin, dtype=float)
    except Exception:
        cal_bin_obs_matrix[run_idx,:] = np.full(len(CAL_PROB_BINS)-1, np.nan)

    # ----------------- Top-K -----------------
    try:
        topk_df = topk_capture(df['y_true'].values, df['y_prob'].values, ks=TOPK_PERCENTS)
        for j,k in enumerate(TOPK_PERCENTS):
            topk_matrix[run_idx,j] = topk_df.loc[topk_df['top_%']==k, 'pos_captured_frac'].values[0]
        temp = topk_df.copy(); temp['run']=run_idx; temp['seed']=seed; topk_rows.append(temp)
    except Exception:
        topk_matrix[run_idx,:] = np.nan
        topk_rows.append(pd.DataFrame())

# ----------------- Aggregation -----------------
metrics_df = pd.DataFrame(metrics_rows)
metrics_df.to_csv(METRICS_PER_RUN_CSV, index=False)

def agg_stats(series):
    mean = np.nanmean(series)
    sd = np.nanstd(series, ddof=1) if np.sum(~np.isnan(series))>1 else np.nan
    n = np.sum(~np.isnan(series))
    se = sd / math.sqrt(n) if n>0 and not np.isnan(sd) else np.nan
    ci95 = 1.96 * se if se is not None else np.nan
    return mean, sd, ci95

agg_rows = []
metrics_to_agg = ["roc_auc","pr_auc","brier","f1_at_0.5","precision_at_0.5","recall_at_0.5","positive_rate"]
for metric in metrics_to_agg:
    mean, sd, ci95 = agg_stats(metrics_df[metric].values)
    agg_rows.append({"stratum": "ALL", "metric": metric, "mean": mean, "sd": sd, "ci95": ci95})

if stratum_col:
    strata = df_orig[stratum_col].dropna().unique().tolist()
    for stratum_val in strata:
        per_stratum_metrics = []
        for run_idx, seed in enumerate(seeds):
            np.random.seed(seed)
            if SAMPLING_METHOD == 'stratified_bootstrap':
                idx_list = []
                for cls in df_orig['y_true'].unique():
                    cls_idx = df_orig.index[df_orig['y_true']==cls].tolist()
                    if len(cls_idx)==0: continue
                    s = np.random.choice(cls_idx, size=len(cls_idx), replace=True)
                    idx_list.extend(s.tolist())
                sampled_idx = np.array(idx_list)
            else:
                n_take = int(math.ceil(n_total * TEST_SAMPLE_FRACTION))
                sampled_idx = np.random.choice(df_orig.index, size=n_take, replace=False)
            df_run_stratum = df_orig.loc[sampled_idx][df_orig[stratum_col]==stratum_val]
            if len(df_run_stratum)==0:
                per_stratum_metrics.append({m: np.nan for m in metrics_to_agg})
                continue
            try: roc_auc = roc_auc_score(df_run_stratum['y_true'], df_run_stratum['y_prob'])
            except Exception: roc_auc = np.nan
            try: pr_auc = average_precision_score(df_run_stratum['y_true'], df_run_stratum['y_prob'])
            except Exception: pr_auc = np.nan
            brier = brier_score_loss(df_run_stratum['y_true'], df_run_stratum['y_prob'])
            y_pred = (df_run_stratum['y_prob']>=0.5).astype(int)
            per_stratum_metrics.append({
                "roc_auc": roc_auc, "pr_auc": pr_auc, "brier": brier,
                "f1_at_0.5": f1_score(df_run_stratum['y_true'], y_pred, zero_division=0),
                "precision_at_0.5": precision_score(df_run_stratum['y_true'], y_pred, zero_division=0),
                "recall_at_0.5": recall_score(df_run_stratum['y_true'], y_pred, zero_division=0),
                "positive_rate": float(df_run_stratum['y_true'].mean())
            })
        per_stratum_df = pd.DataFrame(per_stratum_metrics)
        for metric in metrics_to_agg:
            mean, sd, ci95 = agg_stats(per_stratum_df[metric].values)
            agg_rows.append({"stratum": str(stratum_val), "metric": metric, "mean": mean, "sd": sd, "ci95": ci95})

agg_df = pd.DataFrame(agg_rows)
agg_df.to_csv(METRICS_AGG_CSV, index=False)

# ----------------- Top-K aggregation -----------------
topk_all = pd.concat([t.assign(run=int(t['run'].iloc[0])) if not t.empty else pd.DataFrame() for t in topk_rows], ignore_index=True, sort=False)
if not topk_all.empty: topk_all.to_csv(TOPK_PER_RUN_CSV, index=False)
topk_agg = [{"top_%": k, "mean_frac": agg_stats(topk_matrix[:,j])[0],
             "sd": agg_stats(topk_matrix[:,j])[1], "ci95": agg_stats(topk_matrix[:,j])[2]}
            for j,k in enumerate(TOPK_PERCENTS)]
pd.DataFrame(topk_agg).to_csv(TOPK_AGG_CSV, index=False)

# ----------------- Plots (ROC, PR, Top-K, Reliability) -----------------
# Convert to arrays
roc_arr = np.array(roc_list)
pr_arr = np.array(pr_list)
cal_arr = np.array(cal_list)
topk_arr = np.array(topk_list)
metrics_df = pd.DataFrame(metrics_all)

# ----------------- Plotting with ribbons -----------------
ROC_FPR_GRID = np.linspace(0,1,100)
PR_RECALL_GRID = np.linspace(0,1,100)
TOPK_PERCENTS = np.linspace(0.01,1,100)
CAL_PROB_BINS = np.linspace(0,1,11)  # 10 bins

import matplotlib.pyplot as plt
import numpy as np
import os

# ---------- ROC Curve ----------
mean_tpr = np.nanmean(roc_arr, axis=0)
sd_tpr = np.nanstd(roc_arr, axis=0, ddof=1)

plt.figure(figsize=(6,5))
plt.plot(ROC_FPR_GRID, mean_tpr, label=f"Mean ROC (n={NUM_RUNS})", linewidth=2)
plt.fill_between(ROC_FPR_GRID, mean_tpr - sd_tpr, mean_tpr + sd_tpr, alpha=0.2)
plt.plot([0,1],[0,1], linestyle='--', color='gray', label='Random')

plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('ROC Curve (mean ± SD)', fontsize=18)
plt.legend(loc='lower right', fontsize=18)
plt.grid(alpha=0.3)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'roc_curve_mean_sd.png'), dpi=200)
plt.close()


# ---------- PR Curve ----------
mean_prec = np.nanmean(pr_arr, axis=0)
sd_prec = np.nanstd(pr_arr, axis=0, ddof=1)

plt.figure(figsize=(6,5))
plt.plot(PR_RECALL_GRID, mean_prec, label=f"Mean PR (n={NUM_RUNS})", linewidth=2)
plt.fill_between(PR_RECALL_GRID, mean_prec - sd_prec, mean_prec + sd_prec, alpha=0.2)

plt.xlabel('Recall', fontsize=18)
plt.ylabel('Precision', fontsize=18)
plt.title('Precision-Recall Curve (mean ± SD)', fontsize=18)
plt.legend(loc='upper right', fontsize=18)
plt.grid(alpha=0.3)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'pr_curve_mean_sd.png'), dpi=200)
plt.close()


# ---------- Top-K Capture Plot ----------
n_topk = topk_matrix.shape[1]
x_vals = TOPK_PERCENTS if 'TOPK_PERCENTS' in globals() and len(TOPK_PERCENTS)==n_topk else np.linspace(0, 50, n_topk)
mean_topk = np.nanmean(topk_matrix, axis=0)
sd_topk = np.nanstd(topk_matrix, axis=0, ddof=1)

plt.figure(figsize=(6,4))
plt.plot(x_vals, mean_topk, marker='o', color='tab:blue', linewidth=2, label='Mean Top-K capture')
plt.fill_between(x_vals, mean_topk - sd_topk, mean_topk + sd_topk, color='tab:blue', alpha=0.2, label='± SD')

plt.xlabel('Top-k percent of highest risk area', fontsize=18)
plt.ylabel('Fraction of fires captured', fontsize=18)
plt.title('Top-K Capture (mean ± SD)', fontsize=18)
plt.ylim(0,1)
plt.xlim(0,50)
plt.grid(alpha=0.3)
plt.legend(loc='lower right', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'topk_curve_mean_sd.png'), dpi=200)
plt.close()


# ---------- Reliability / Calibration ----------
cal_bin_centers = (CAL_PROB_BINS[:-1] + CAL_PROB_BINS[1:]) / 2.0
mean_bin_obs = np.nanmean(cal_arr, axis=0)
sd_bin_obs = np.nanstd(cal_arr, axis=0, ddof=1)

plt.figure(figsize=(6,5))
plt.plot(cal_bin_centers, mean_bin_obs, marker='o', linewidth=2, label='Mean calibration')
plt.fill_between(cal_bin_centers, mean_bin_obs - sd_bin_obs, mean_bin_obs + sd_bin_obs, alpha=0.2)
plt.plot([0,1],[0,1], linestyle='--', color='gray', label='Perfect')

plt.xlabel('Predicted probability (bin center)', fontsize=16)
plt.ylabel('Observed frequency', fontsize=16)
plt.title('Reliability Diagram (mean ± SD)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(alpha=0.3)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,'reliability_mean_sd.png'), dpi=200)
plt.close()


# ----------------- Summary Text -----------------
report_lines = [
    "Region-transfer / Temporal-style Multi-run Evaluation",
    f"File: {csv_path}",
    f"Total original samples: {n_total}, Overall positive rate: {pos_rate_total:.4f}",
    f"Sampling method: {SAMPLING_METHOD}",
    f"Number of runs (K): {K_RUNS}",
    f"Base seed (document for reproducibility): {BASE_SEED}",
    f"Seeds used: {seeds}",
    "",
    f"Metrics per-run saved to: {METRICS_PER_RUN_CSV}",
    f"Aggregated metrics saved to: {METRICS_AGG_CSV} (mean ± sd ± 95%CI)",
    f"Top-K per-run saved to: {TOPK_PER_RUN_CSV}",
    f"Top-K aggregated saved to: {TOPK_AGG_CSV}",
    f"ROC plot (mean ± SD): {OUT_DIR / 'roc_curve_mean_sd.png'}",
    f"PR plot (mean ± SD): {OUT_DIR / 'pr_curve_mean_sd.png'}",
    f"Reliability plot (mean ± SD): {OUT_DIR / 'reliability_mean_sd.png'}",
    f"Top-K plot (mean ± SD): {OUT_DIR / 'topk_curve_mean_sd.png'}",
]
with open(SUMMARY_TXT, 'w') as f:
    f.write("\n".join(report_lines))

print("\n".join(report_lines))
print("Outputs written to:", OUT_DIR)


Using CSV: /content/Validation Datasets/RF_TemporalSplit_Test2025.csv
Detected columns -> label: class , prob: classification
No stratum column detected - will evaluate overall only.
Running 10 runs with seeds: [1281540326, 1005233768, 2011547310, 1367058049, 1542280147, 90017157, 581114276, 1258272007, 2134214070, 1923482161]
Region-transfer / Temporal-style Multi-run Evaluation
File: /content/Validation Datasets/RF_TemporalSplit_Test2025.csv
Total original samples: 2148, Overall positive rate: 0.7658
Sampling method: stratified_bootstrap
Number of runs (K): 10
Base seed (document for reproducibility): 20250908
Seeds used: [1281540326, 1005233768, 2011547310, 1367058049, 1542280147, 90017157, 581114276, 1258272007, 2134214070, 1923482161]

Metrics per-run saved to: //content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/forest/Results/results_region_transfer_V5/metrics_per_run.csv
Aggregated metrics saved to: //content/drive/MyDrive/California_Fire_MS/Fire_Risk_Validation/fore

In [ ]:
# Use gdown to download the folder
!gdown "https://drive.google.com/drive/u/0/folders/1BjUySwa9W__GIX4arSDQHRHT8brUGjap" --folder

Retrieving folder contents
Retrieving folder 1rKgmkgPXAdch0kSCMlG5BiABuTHVq15r results_region_transfer
Processing file 1MAHhVOjdidwadwULttFfOigE0vMVWc48 metrics_per_run.csv
Processing file 1Vzv-05tDwKm09ftsZcmEfNrLmVmKxKHo pr_curve_mean_sd.png
Processing file 13U6alSSJgDx1veMaubIOFvVTpV6zDUf0 reliability_mean_sd.png
Processing file 1DypI6QnzOaGZabZClmWax4WuEKaOqglZ rf_model_42.joblib
Processing file 1EvoUeHw8HXHeJg1IWCKOFvNxpzDysRrq rf_model_43.joblib
Processing file 19wfT4MGPmiKcaD35NFhggCWINn4mW_6g rf_model_44.joblib
Processing file 1U5nUAdQtDEJuVv2GaELyVtw1MhIsSMw3 rf_model_45.joblib
Processing file 1VYQKWS63YaAEuEDpIRUHrvaIUJg_N46B rf_model_46.joblib
Processing file 1dNqN9B8qUK9PhFsTp9OliZ4OOLlo5VvY rf_model_47.joblib
Processing file 1DpLUEKAoYzWlDC5Uqrg1pJvbucK8wzyO rf_model_48.joblib
Processing file 1cHajbGu7cavQN8vSU_YzZRNaMSnOmgtw rf_model_49.joblib
Processing file 1zg7rYGpfc1oHyiELqDoPJvl6kG4W7n7H rf_model_50.joblib
Processing file 1zsK_JXDJ07v6dKCjWqcE8nleKTeo6tYe rf_model_51